In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [2]:
df = pd.read_csv('../capstone-data/daily/CTA_-_Ridership_-_Daily_Boarding_Totals.csv')
gas = pd.read_csv('../capstone-data/weekly/Weekly_Chicago_Regular_All_Formulations_Retail_Gasoline_Prices.csv')
covid = pd.read_csv('../capstone-data/monthly/covid-restrictions.csv')
vax= pd.read_csv('../capstone-data/covid/COVID-19_Daily_Vaccinations_-_Chicago_Residents.csv')
weather = pd.read_csv('../capstone-data/daily/weather.csv')
curve = pd.read_excel('../capstone-data/daily/treasury_yield_curve.xlsx')

In [3]:
df.head()

,service_date,day_type,bus,rail_boardings,total_rides
0,01/01/2001,U,297192,126455,423647
1,01/02/2001,W,780827,501952,1282779
2,01/03/2001,W,824923,536432,1361355
3,01/04/2001,W,870021,550011,1420032
4,01/05/2001,W,890426,557917,1448343


In [4]:
df.shape

(7701, 5)

In [5]:
df.dtypes

service_date      object
day_type          object
bus                int64
rail_boardings     int64
total_rides        int64
dtype: object

In [6]:
df.drop(columns={'day_type'},inplace=True)

In [7]:
df.rename(columns={'service_date':'date'},inplace=True)
df['date']= pd.to_datetime(df['date'])

In [8]:
df.head()

,date,bus,rail_boardings,total_rides
0,2001-01-01,297192,126455,423647
1,2001-01-02,780827,501952,1282779
2,2001-01-03,824923,536432,1361355
3,2001-01-04,870021,550011,1420032
4,2001-01-05,890426,557917,1448343


In [9]:
df.set_index('date',inplace=True) 
df.sort_index(inplace=True)

In [10]:
#df1= df.groupby(df['date']).agg('sum')
weekly = df.resample('W',closed='left').sum()
weekly.index = weekly.index - pd.DateOffset(days=6)

In [11]:
weekly.shape

(1092, 3)

In [12]:
weekly

,bus,rail_boardings,total_rides
date,,,
2001-01-01,4240790,2528123,6768913
2001-01-08,5930650,3445299,9375949
2001-01-15,5591301,3295995,8887296
2001-01-22,5956378,3463065,9419443
2001-01-29,5936801,3436499,9373300
...,...,...,...
2021-11-01,2747792,2013070,4760862
2021-11-08,2604481,1951484,4555965
2021-11-15,2647447,2007509,4654956


## Prep Gas

In [16]:
gas = gas.rename(columns={'Week of':'date', 'Weekly Chicago Regular All Formulations Retail Gasoline Prices Dollars per Gallon':'price'})

In [17]:
gas['date']=pd.to_datetime(gas['date'])

In [18]:
gas.set_index('date',inplace=True)
gas.sort_index(inplace=True)
gas

,price
date,
2000-06-05,1.949
2000-06-12,2.091
2000-06-19,2.108
2000-06-26,1.989
2000-07-03,1.786
...,...
2021-12-27,3.340
2022-01-03,3.320
2022-01-10,3.457


In [19]:
merged = weekly.merge(gas, how='outer', left_index=True, right_index=True)

In [20]:
merged.isna().sum()

bus               38
rail_boardings    38
total_rides       38
price              0
dtype: int64

In [21]:
merged.dropna(inplace=True)

In [23]:
merged.shape

(1092, 4)

In [27]:
merged

,bus,rail_boardings,total_rides,price
date,,,,
2001-01-01,4240790.0,2528123.0,6768913.0,1.487
2001-01-08,5930650.0,3445299.0,9375949.0,1.477
2001-01-15,5591301.0,3295995.0,8887296.0,1.569
2001-01-22,5956378.0,3463065.0,9419443.0,1.546
2001-01-29,5936801.0,3436499.0,9373300.0,1.541
...,...,...,...,...
2021-11-01,2747792.0,2013070.0,4760862.0,3.539
2021-11-08,2604481.0,1951484.0,4555965.0,3.616
2021-11-15,2647447.0,2007509.0,4654956.0,3.557


## Weather

In [ ]:
# df.to_csv('../capstone-data/cta-monthly-primary.csv')